In [3]:
import lightfm
import scipy as sp
import pymongo
import pandas as pd
import numpy as np

c:\Users\Cody\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [4]:
# Read in yelp data from mongodb
import pymongo
import pandas as pd
import numpy as np

# # Connect to mongodb on 192.168.1.87
# client = pymongo.MongoClient(
#     host='192.168.1.87',
#     port=27017)
# # Get the yelp database
# db = client.yelp
# review = db.review

# # Get a random sample of 1000 reviews
# pipeline = [
#     {
#         '$project': {
#             '_id': 0, 
#             'user_id': '$user_id', 
#             'business_id': '$business_id', 
#             'stars': '$stars'
#         }
#     }, 
#     # {
#     #     '$sample': {
#     #         'size': 10000
#     #     }
#     # }
# ]
# df = pd.DataFrame(list(review.aggregate(pipeline)))
# # Subset to only the user_id, business_id, and stars
# df = df[['user_id', 'business_id', 'stars']]
# # Write to parquet to save space
# df.to_parquet('yelp.parquet',compression='gzip')

In [14]:
# Read in yelp.parquet
df = pd.read_parquet('yelp.parquet')
# Sample df down to 10000 rows
df = df.sample(100000)
# Train test split
from sklearn.model_selection import train_test_split
# Replace user_id and business_id with unique integers
user_id = df.user_id.unique()
business_id = df.business_id.unique()
user_id_map = {user_id[i]: i for i in range(len(user_id))}
business_id_map = {business_id[i]: i for i in range(len(business_id))}
# Replace
df.user_id = df.user_id.map(user_id_map)
df.business_id = df.business_id.map(business_id_map)
# Replace NaN with 0
df.stars = df.stars.fillna(0)
# Add column to indicate train test split
df['train'] = np.random.uniform(0, 1, len(df)) <= .75
# Create sparse matrix of user_id, business_id, stars
# That is size  (no_users, no_businesses)
train = df[df.train==True]
test = df[df.train==False]
train_sparse = sp.sparse.coo_matrix((train.stars, (train.user_id, train.business_id))) 
test_sparse = sp.sparse.coo_matrix((test.stars, (test.user_id, test.business_id)))
# Print train and test dimensions
print(f'Train shape: {train_sparse.shape}')
print(f'Test shape: {test_sparse.shape}')

Train shape: (84163, 46287)
Test shape: (84164, 46285)


In [15]:
# Each row represents a user, and each column an item. Entries are ratings from 1 to 5. 

In [16]:
# Get unique number of users and businesses in train (not train_sparse)
n_users_train = len(df[df.train].user_id.unique())
n_businesses_train = len(df[df.train].business_id.unique())
print(f'Train users: {n_users_train}\t Train businesses: {n_businesses_train}')
# Same for test
n_users_test = len(df[~df.train].user_id.unique())
n_businesses_test = len(df[~df.train].business_id.unique())
print(f'Test users: {n_users_test}\t Test businesses: {n_businesses_test}')
# Do for total dataset
n_users = len(df.user_id.unique())
n_businesses = len(df.business_id.unique())
print(f'Total users: {n_users}\t Total businesses: {n_businesses}')

Train users: 65047	 Train businesses: 38802
Test users: 23411	 Test businesses: 17637
Total users: 84164	 Total businesses: 46287


In [19]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

# Instantiate and train the model
model = LightFM()
model.fit(train_sparse, epochs=30, num_threads=2)

In [20]:
test_precision = precision_at_k(model, train_sparse, k=5).mean()
print(f'Train Precision: {test_precision:.2f}')


In [ ]:

train_auc = auc_score(model, train_sparse).mean()
print(f'Train AUC: {train_auc:.2f}')

Train AUC: 0.9138168692588806
